In [1]:
!pip install --quiet --upgrade openai-agents neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.5/288.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.0 MB/s eta 0:00:00


The **OpenAI Agents SDK for Python** is OpenAI's toolkit for building AI agents. Built on a lightweight, flexible architecture, it provides native tool integration, seamless handoffs between agents, guardrails for input/output validation, and production essentials like tracing, error handling, and session management.

In this notebook, we'll build a company research agent that queries a Neo4j graph database.

In [19]:
import base64
import json
import os

from agents import Agent, Runner, ModelSettings, function_tool
from agents.mcp import MCPServerStreamableHttp

from typing import Any

The OpenAI Agent SDK is provider-agnostic, supporting other LLMs via the LiteLLM integration.

Here we will be using the OpenAI API.

In [2]:
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key")

OpenAI API key··········


First, we define a `pretty_print` function to execute queries and display the agent's step-by-step reasoning, tool calls, and results.

In [30]:
def pretty_print(result):
    for i, item in enumerate(result.new_items):
        print(f"\n[Step {i + 1}] {item.type}")
        print("-" * 40)

        if item.type == "tool_call_item":
            print(f"  🔧 Tool: {item.raw_item.name}")
            print(f"  📥 Arguments:")
            try:
                args = json.loads(item.raw_item.arguments)
                print(f"     {json.dumps(args, indent=6)}")
            except:
                print(f"     {item.raw_item.arguments}")

        elif item.type == "tool_call_output_item":
            print(f"  📤 Output:")
            try:
                output = json.loads(item.output)
                print(f"     {json.dumps(output, indent=6)}")
            except:
                print(f"     {item.output}")

        elif item.type == "message_output_item":
            print(f"  💬 Message:")
            for content in item.raw_item.content:
                if hasattr(content, "text"):
                    print(f"     {content.text}")

## Neo4j MCP Server

We'll start by using the official [Neo4j MCP Server](https://github.com/neo4j/mcp) to extend the agent with Neo4j tools. This MCP server provides the agent with capabilities to read the graph schema and execute Cypher queries, enabling it to fetch and analyze data directly from the database.

The following code installs the latest version on Google Colab and similar Linux-based systems. For other operating systems, please consult the [official installation documentation](https://neo4j.com/docs/mcp/current/installation/).

In [3]:
import requests

# Get latest release info from GitHub API
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Latest version: {version}")

# Download the latest Linux binary
!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz

# Extract
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz

# Make executable
!chmod +x neo4j-mcp

# Cleanup
!rm neo4j-mcp_Linux_x86_64.tar.gz

# Move
!mv neo4j-mcp /usr/local/bin/

# Verify installation
!neo4j-mcp -v


Latest version: v1.2.0
neo4j-mcp version: v1.2.0


For this example, we'll use the companies database from the Neo4j demo server, which contains organizations, people, investors, and news articles.

For HTTP transport, you only need to set the `NEO4J_URI` and optionally `NEO4J_DATABASE` (if connecting to a specific database).

In [4]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_DATABASE"] = "companies"
os.environ["NEO4J_MCP_TRANSPORT"] = "http"

# Run the server in the background
import subprocess
subprocess.Popen(["neo4j-mcp"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

<Popen: returncode: None args: ['neo4j-mcp']>

Here we create an MCP client that connects to our Neo4j server endpoint using `MCPServerStreamableHttp`. Credentials are passed via the `Authorization` header using Basic authentication. The HTTP transport listens on port 80 by default, so the MCP endpoint is available at `http://localhost:80/mcp`. We then pass the server directly to the `Agent` via the `mcp_servers` parameter, which automatically discovers and exposes all tools from the MCP server to the agent.

In [12]:
# Credentials are passed via bearer auth
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"

credentials = base64.b64encode(f"{os.environ["NEO4J_USERNAME"]}:{os.environ["NEO4J_PASSWORD"]}".encode()).decode()

async with MCPServerStreamableHttp(
    name="Neo4j server",
    params={
        "url": "http://localhost:80/mcp",
        "headers": {"Authorization": f"Basic {credentials}"},
        "timeout": 10,
    },
) as server:
    agent = Agent(
        name="Assistant",
        instructions="""Use the MCP tools to answer the questions.
        Always use read-schema first to evaluate the graph schema if you want to execute any cypher queries.""",
        mcp_servers=[server],
        model_settings=ModelSettings(tool_choice="required"),
    )

    result = await Runner.run(agent, "How many people are in the database?")
    pretty_print(result)




[Step 1] tool_call_item
----------------------------------------
  🔧 Tool: get-schema
  📥 Arguments:
     {}

[Step 2] tool_call_output_item
----------------------------------------
  📤 Output:
     {
      "type": "text",
      "text": "[{\"key\":\"_Bloom_Perspective_\",\"value\":{\"type\":\"node\",\"properties\":{\"data\":\"STRING\",\"id\":\"STRING\",\"name\":\"STRING\",\"roles\":\"LIST\",\"version\":\"STRING\"},\"relationships\":{\"_Bloom_HAS_SCENE_\":{\"direction\":\"out\",\"labels\":[\"_Bloom_Scene_\"]}}}},{\"key\":\"HAS_CEO\",\"value\":{\"type\":\"relationship\"}},{\"key\":\"Organization\",\"value\":{\"type\":\"node\",\"properties\":{\"diffbotId\":\"STRING\",\"id\":\"STRING\",\"isDissolved\":\"BOOLEAN\",\"isPublic\":\"BOOLEAN\",\"motto\":\"STRING\",\"name\":\"STRING\",\"nbrEmployees\":\"INTEGER\",\"revenue\":\"FLOAT\",\"summary\":\"STRING\"},\"relationships\":{\"HAS_BOARD_MEMBER\":{\"direction\":\"out\",\"labels\":[\"Person\"]},\"HAS_CATEGORY\":{\"direction\":\"out\",\"labels\":

## Custom Tools
Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case.

First, we need to define a Neo4j driver to establish a connection to the database and an `execute_cypher` helper method to run queries.

In [15]:
from neo4j import AsyncGraphDatabase

# Create async driver
driver = AsyncGraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))


async def execute_cypher(cypher: str, params: dict) -> list[dict]:
    result = await driver.execute_query(cypher, database_=os.environ["NEO4J_DATABASE"], **params)
    return [record.data() for record in result.records]


The first custom tool uses a fulltext index to search for companies by name. The `@function_tool` decorator automatically generates the tool's name and input schema from the function signature, while the docstring provides the description that the agent uses to understand when and how to call it.

In [25]:
@function_tool
async def find_companies(company_name: str) -> str:
    "List of Companies (company_id, name, summary) by fulltext search"
    query = """
        CALL db.index.fulltext.queryNodes('entity', $search, {limit: 100})
        YIELD node as c, score WHERE c:Organization
        AND NOT EXISTS { (c)<-[:HAS_SUBSIDIARY]-() }
        RETURN c.id as company_id, c.name as name, c.summary as summary LIMIT 10
    """
    try:
        results = await execute_cypher(query, {"search": company_name})
        return json.dumps(results)
    except Exception as e:
        return f"Error: {e}"

The second tool retrieves investors for a given company, returning their IDs, names, and types (e.g., Organization or Person).

In [27]:
@function_tool
async def get_investors(company: str) -> str:
    "Returns the investments by a company by name. Returns list of investment ids, names and types."
    try:
        return await execute_cypher("""
            MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
            WHERE o.name = $company
            RETURN i.id as id, i.name as name, head(labels(i)) as type
        """, {"company": company})
    except Exception as e:
        return f"Error: {e}"

Now we combine the MCP server with our custom tools by passing both to the agent. The `mcp_servers` parameter provides access to the Neo4j MCP tools, while the `tools` parameter adds our custom `find_companies` and `get_investors` functions. This gives the agent a unified toolkit spanning both MCP-exposed and locally-defined capabilities.

In [29]:
async with MCPServerStreamableHttp(
    name="Neo4j server",
    params={
        "url": "http://localhost:80/mcp",
        "headers": {"Authorization": f"Basic {credentials}"},
        "timeout": 10,
    },
) as server:
    agent = Agent(
        name="Assistant",
        instructions="""Use the MCP tools to answer the questions.
        Always use read-schema first to evaluate the graph schema if you want to execute any cypher queries.""",
        mcp_servers=[server],
        tools=[find_companies, get_investors],  # Add your tools here
        model_settings=ModelSettings(tool_choice="required"),
    )

    result = await Runner.run(agent, "Which companies has Google invested in?")
    pretty_print(result)


[Step 1] tool_call_item
----------------------------------------
  🔧 Tool: get_investors
  📥 Arguments:
     {
      "company": "Google"
}

[Step 2] tool_call_output_item
----------------------------------------
  📤 Output:
     [{'id': 'ELsv5bECSOiWG_Uhf_txI2w', 'name': 'Ionic Security', 'type': 'Organization'}, {'id': 'EUkm62r-bMOidNtPjTkdVvg', 'name': 'Avere Systems', 'type': 'Organization'}, {'id': 'EX-RLztfkOFqTLoM6xIVnlg', 'name': 'FlexiDAO', 'type': 'Organization'}, {'id': 'EtqXbQ9LaMGq8om4dhYY0Fw', 'name': 'Cloudflare', 'type': 'Organization'}, {'id': 'EWIvDLNCSMCCBYUyz0oFPVQ', 'name': 'Trifacta', 'type': 'Organization'}]

[Step 3] message_output_item
----------------------------------------
  💬 Message:
     Google has invested in the following companies:

1. Ionic Security
2. Avere Systems
3. FlexiDAO
4. Cloudflare
5. Trifacta

If you need more details about any of these companies or their investment rounds, let me know!


## Summary

In this notebook, we built a company research agent using the **OpenAI Agent SDK** with Neo4j:

1. **MCP Integration** — Connected to Neo4j using the official Neo4j MCP Server via HTTP transport, enabling schema reading and Cypher query execution

2. **Custom Tools** — Created specialized tools (`find_companies`, `get_investors`) using the `@function_tool` decorator to perform fulltext search and retrieve investment relationships

The OpenAI Agent SDK makes it straightforward to integrate external MCP servers, define custom tools via simple decorated functions, and build agents that can query graph databases to answer natural language questions.